In [1]:
import sys
import genlib

In [2]:
#npop = 50
#evo = genlib.Evolution(npop, npcs=13, mut_threshold=0.01, instance=0)
#evo.darwinize()

- check rotation, mirroring
- add checkers

In [ ]:
while True:
    ind = genlib.Individual([12,0,10,9,6,8,3,5,7,2,1,11,4])
    if ind.fitness == 0:
        print(ind.placements)
        break

Exception ignored in: <function BaseGeometry.__del__ at 0x7f26b6f620e0>
Traceback (most recent call last):
  File "/home/mlubej/.pyenv/versions/3.7.6/envs/play/lib/python3.7/site-packages/shapely/geometry/base.py", line 239, in __del__
    def __del__(self):
KeyboardInterrupt


In [ ]:
def opt_placement(profile, poly):
    mask = np.array([pt.within(profile) for pt in pt_grid], dtype=bool)

    rflips = get_unique_rflips(poly)
    pos = pos_grid[mask]

    iterables = [pos, rflips]
    settings = np.array(list(itertools.product(*iterables)))
    settings = np.array([[s[0], *s[1]] for s in settings]) 
    
    scores, profiles = np.array([calc_score(profile, poly, s) for s in settings]).T
    opt_id = np.argsort(scores)
    return settings[opt_id], profiles[opt_id]